<a href="https://colab.research.google.com/github/yojulab/learn_data_analytics/blob/main/codes/cases/R04_Regression_pipeline_Manufacture_Wafers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd # import: 라이브러리를 해당 스크립트로 호출하는 함수
df1 = pd.read_csv('preprocessing_data.csv')
# read csv : csv 파일을 불러오는 함수
df1.head(6) # head( ) : 상위 7개의 데이터를 확인

In [ ]:
Y = df1['Target'] # Wafer W #8 chip +
X = df1[['Temp_OXid', 'Oxid_time', 'thickness', 'Line_CD', 'Etching_rate', 'Temp_Etching']]
X.head(5)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, random_state=1234)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)


In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [ ]:
def evaluation_reg(model):
    Y_train_pred = model.predict(X_train)
    Y_test_pred = model.predict(X_test)
    print('학습 데이터 결정계수: ', r2_score(Y_train, Y_train_pred))
    print('검증 데이터 결정계수: ',r2_score(Y_test, Y_test_pred))
    print('학습 데이터 MSE :', mean_squared_error(Y_train, Y_train_pred))
    print('검증 데이터 MSE :', mean_squared_error(Y_test, Y_test_pred))
    print('학습 데이터 MAE : ', mean_absolute_error(Y_train, Y_train_pred))
    print('검증 데이터 MAE : ', mean_absolute_error(Y_test, Y_test_pred))

### 특성 공학 기법 적용 - 결측치 처리
- Pipe Line : 학습과 특성공학 기법을 동시에 사용하여, 데이터를 한꺼번에 처리하면서 학 습을 수행하는 함수


In [ ]:
from sklearn.pipeline import Pipeline # 학습 + 특성공학
from sklearn.impute import SimpleImputer # 결측값 처리 함수
from sklearn.linear_model import LinearRegression

In [ ]:
pipe_list = [('imputer', SimpleImputer() ), ('model', LinearRegression())]
pipe_model = Pipeline(pipe_list)
pipe_model


In [ ]:
pipe_model.fit(X_train, Y_train)
evaluation_reg(pipe_model)

In [ ]:
# **새로운 값을 직접 입력하여 예측**
# x1 = input('Temp_OXid을 입력하시오 :')
# x2 = input('Oxid_time을 입력하시오:')
# x3 = input(' thickness 입력하시오 : ')
# x4 = input(' Line_co 입력하시오 : ')
# x5 = input('Etching_rate 입력하시오 : ')
# x6 = input('Temp_Etching 입력하시오 : ')
x1 = 1138.979159
x2 = 137.0
x3 = 705.471
x4 = 28.063
x5 = 2.74825
x6 = 71.174

input_new_data = pd. DataFrame([[x1,x2,x3,x4, x5, x6]])
pipe_model.predict(input_new_data)

In [ ]:
#**결측값이 있어도, Pipe Line 함수로 인해 오류 없이 예측값이 계산되어 출력**
import numpy as np
input_new_data2 = pd.DataFrame([[x1, np.nan, x3,x4, x5, x6]], columns=X.columns. tolist())
input_new_data2

In [ ]:
pipe_model.predict(input_new_data2)

### 특성 공학 기법 적용 - scaling & encoding
- [youtube - refer tutorial [Python 활용 제조/공정 데이터 마이닝] 14 파이프라인을 이용한 스케일링 및 인코딩 (Pipeline & Scaling Encoding)
](https://youtu.be/5D33ZhcTups)
- Scal ing -> 연속형
- Encoding -> 범주형
- Imputat ion -> 연속/ 범주


In [ ]:
Y = df1['Target']
X = df1[['Oxid_time', 'thickness', 'Vapor', 'resist_target', 'Line_CD', 'Etching_rate', 'UV_type' ]]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split (X,Y, random_state= 1234)

In [ ]:
# 운자데이터와 숫자데이터를 구분하여 처리하는 프로세스를 구축할 때 사용하는 함수
from sklearn.compose import make_column_transformer
# 특성공학 + 학습
from sklearn.pipeline import make_pipeline
# Scaling & Encoding
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
# 연속형/범주형 데이터 항목영을 리스트로 선언
numeric_list = ['Oxid_time', 'thickness', 'resist_target', 'Line_CD', 'Etching_rate']
category_list = ['Vapor', 'UV_type']

In [ ]:
# 연속형 데이터가 처리되는 Process Pipe
# 결측값을 평균으로 대치한 다음, Scale을 최소 0/ 최대 1으로 변환
numeric_pipe = make_pipeline( SimpleImputer(strategy='mean'),  MinMaxScaler () )
numeric_pipe

In [ ]:
# 범주형 데이터가 처리되는 Process Pipe
# 결수값을 최빈값으로 대치한 다음, 문자데이터를 Onatot 1인코딩 변환
category_pipe= make_pipeline( SimpleImputer (strategy='most_frequent'), OneHotEncoder ( ))
category_pipe

In [ ]:
# 연속형과 범주형을 나누어 처리하는 Pipe을 구성
preprocessing_pipe = make_column_transformer ( (numeric_pipe, numeric_list), (category_pipe, category_list) )
preprocessing_pipe

In [ ]:
pipe_model = make_pipeline (preprocessing_pipe, LinearRegression () )
pipe_model.fit (X_train, Y_train)
evaluation_reg(pipe_model)

### 특성공학 - Cross Validation
- [youtube - tutorial [Python 활용 제조/공정 데이터 마이닝] 15 파이프라인을 이용한 교차 검증 (Pipeline & Cross Validation)](https://youtu.be/OH5nEK0b0uA)


In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# 5회 교차검증 실시하며 학습 수행
grid_model = GridSearchCV(pipe_model, param_grid={}, cv=5)
grid_model.fit(X_train,Y_train)


In [ ]:
# 교자검증 이후 가장 성능이 좋은 1agel 선택
best_model = grid_model.best_estimator_
evaluation_reg(best_model)

### Feature Enginnering
- [youtube - tutorial [Python 활용 제조/공정 데이터 마이닝] 16 모델 하이퍼파라미터 튜닝 (Pipeline & Hyper Parameter Tuning)](https://youtu.be/TCC-ZSjAW-c)

In [ ]:
# 의사결정나무 알고리증
from sklearn.tree import DecisionTreeRegressor

In [ ]:
# 연속형과 범주형을 나누어 처리하는 Pipe를 구성
preprocessing_pipe = make_column_transformer ( (numeric_pipe, numeric_list), (category_pipe, category_list) )
pipe_model = make_pipeline (preprocessing_pipe, DecisionTreeRegressor () )
pipe_model

In [ ]:
# deoth : 5~10 5개 / s011t : 1~10 10개 / /eaf : 5~10 5개 -> 250가지 경우의 수
hyperparameter = {'decisiontreeregressor__max_depth': range (5, 11)
                , 'decisiontreeregressor__min_samples_split':range (1, 11)
                , 'decisiontreeregressor__min_samples_leaf':range(5, 11)}
grid_model = GridSearchCV (pipe_model, param_grid=hyperparameter, cv=5)
# 5회의 교차검증 수행
grid_model.fit(X_train, Y_train)

In [ ]:
# 가장 적절한 모델을 선택
best_model = grid_model.best_estimator_
print(best_model)

In [ ]:
evaluation_reg(best_model)